### Block 1: Environment Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("Environment setup complete!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Environment setup complete!
Pandas version: 2.2.3
NumPy version: 2.2.2


### Block 2: Load Processed Files

In [2]:
# Load processed files from Notebook 1
import os

processed_path = "data/processed/"

print("Loading processed files from Phase A...")

# Load beneficiary files
print("Loading beneficiary files...")
df_ben_2008 = pd.read_parquet(f"{processed_path}beneficiary_2008_processed.parquet")
df_ben_2009 = pd.read_parquet(f"{processed_path}beneficiary_2009_processed.parquet") 
df_ben_2010 = pd.read_parquet(f"{processed_path}beneficiary_2010_processed.parquet")

# Load inpatient claims
print("Loading inpatient claims...")
df_inpatient = pd.read_parquet(f"{processed_path}inpatient_claims_processed.parquet")

print("✓ All files loaded successfully!")
print(f"├── Beneficiary 2008: {df_ben_2008.shape[0]:,} rows")
print(f"├── Beneficiary 2009: {df_ben_2009.shape[0]:,} rows") 
print(f"├── Beneficiary 2010: {df_ben_2010.shape[0]:,} rows")
print(f"└── Inpatient Claims: {df_inpatient.shape[0]:,} rows")

Loading processed files from Phase A...
Loading beneficiary files...
Loading inpatient claims...
✓ All files loaded successfully!
├── Beneficiary 2008: 116,352 rows
├── Beneficiary 2009: 114,538 rows
├── Beneficiary 2010: 112,754 rows
└── Inpatient Claims: 66,773 rows


### Block 3: Beneficiary Data Analysis Before Combination

In [3]:
print("BENEFICIARY DATA ANALYSIS BEFORE COMBINATION")
print("="*50)

# Analyze beneficiary overlap across years
ben_2008_ids = set(df_ben_2008['DESYNPUF_ID'])
ben_2009_ids = set(df_ben_2009['DESYNPUF_ID'])
ben_2010_ids = set(df_ben_2010['DESYNPUF_ID'])

print("BENEFICIARY OVERLAP ANALYSIS:")
print(f"2008 only: {len(ben_2008_ids - ben_2009_ids - ben_2010_ids):,}")
print(f"2009 only: {len(ben_2009_ids - ben_2008_ids - ben_2010_ids):,}")
print(f"2010 only: {len(ben_2010_ids - ben_2008_ids - ben_2009_ids):,}")
print(f"All 3 years: {len(ben_2008_ids & ben_2009_ids & ben_2010_ids):,}")
print(f"2008-2009 only: {len((ben_2008_ids & ben_2009_ids) - ben_2010_ids):,}")
print(f"2009-2010 only: {len((ben_2009_ids & ben_2010_ids) - ben_2008_ids):,}")

# Check for deaths (beneficiaries who disappear)
deaths_2008 = ben_2008_ids - ben_2009_ids
deaths_2009 = ben_2009_ids - ben_2010_ids

print(f"\nDEATH ANALYSIS:")
print(f"Beneficiaries who died in 2008: {len(deaths_2008):,}")
print(f"Beneficiaries who died in 2009: {len(deaths_2009):,}")

# Verify death records
death_records_2008 = df_ben_2008[df_ben_2008['BENE_DEATH_DT'].notna()]
death_records_2009 = df_ben_2009[df_ben_2009['BENE_DEATH_DT'].notna()]
death_records_2010 = df_ben_2010[df_ben_2010['BENE_DEATH_DT'].notna()]

print(f"Death records in 2008 file: {len(death_records_2008):,}")
print(f"Death records in 2009 file: {len(death_records_2009):,}")
print(f"Death records in 2010 file: {len(death_records_2010):,}")

BENEFICIARY DATA ANALYSIS BEFORE COMBINATION
BENEFICIARY OVERLAP ANALYSIS:
2008 only: 1,814
2009 only: 0
2010 only: 0
All 3 years: 112,754
2008-2009 only: 1,784
2009-2010 only: 0

DEATH ANALYSIS:
Beneficiaries who died in 2008: 1,814
Beneficiaries who died in 2009: 1,784
Death records in 2008 file: 1,814
Death records in 2009 file: 1,784
Death records in 2010 file: 1,863


### Block 4: Create Master Beneficiary Dataset

In [4]:
def create_master_beneficiary_dataset():
    """
    Combine beneficiary files from 2008-2010 into master dataset
    Handle deaths and create longitudinal beneficiary records
    """
    print("CREATING MASTER BENEFICIARY DATASET")
    print("="*40)
    
    # Add year indicators
    df_2008 = df_ben_2008.copy()
    df_2009 = df_ben_2009.copy() 
    df_2010 = df_ben_2010.copy()
    
    df_2008['BENE_YEAR'] = 2008
    df_2009['BENE_YEAR'] = 2009
    df_2010['BENE_YEAR'] = 2010
    
    print(f"Adding year indicators...")
    print(f"├── 2008: {len(df_2008)} records")
    print(f"├── 2009: {len(df_2009)} records")
    print(f"└── 2010: {len(df_2010)} records")
    
    # Combine all years
    df_master = pd.concat([df_2008, df_2009, df_2010], ignore_index=True)
    
    print(f"\nCombined dataset: {len(df_master):,} total records")
    print(f"Unique beneficiaries: {df_master['DESYNPUF_ID'].nunique():,}")
    
    # Sort by beneficiary and year
    df_master = df_master.sort_values(['DESYNPUF_ID', 'BENE_YEAR'])
    
    return df_master

# Create master dataset
df_beneficiary_master = create_master_beneficiary_dataset()

# Display sample
print(f"\nSAMPLE OF MASTER BENEFICIARY DATASET:")
print(df_beneficiary_master[['DESYNPUF_ID', 'BENE_YEAR', 'BENE_DEATH_DT', 'SP_CHF', 'SP_DIABETES']].head(10))

CREATING MASTER BENEFICIARY DATASET
Adding year indicators...
├── 2008: 116352 records
├── 2009: 114538 records
└── 2010: 112754 records

Combined dataset: 343,644 total records
Unique beneficiaries: 116,352

SAMPLE OF MASTER BENEFICIARY DATASET:
             DESYNPUF_ID  BENE_YEAR  BENE_DEATH_DT  SP_CHF  SP_DIABETES
0       00013D2EFD8E45D1       2008            NaN   False        False
116352  00013D2EFD8E45D1       2009            NaN   False        False
230890  00013D2EFD8E45D1       2010            NaN    True        False
1       00016F745862898F       2008            NaN   False        False
116353  00016F745862898F       2009            NaN   False         True
230891  00016F745862898F       2010            NaN    True         True
2       0001FDD721E223DC       2008            NaN   False        False
116354  0001FDD721E223DC       2009            NaN   False        False
230892  0001FDD721E223DC       2010            NaN   False        False
3       00021CA6FF03E670       20

### Block 5: Handle Missing Values Strategy

In [5]:
def analyze_missing_values(df, dataset_name):
    """
    Comprehensive missing value analysis
    """
    print(f"\nMISSING VALUES ANALYSIS - {dataset_name}")
    print("="*50)
    
    # Calculate missing values
    missing_counts = df.isnull().sum()
    missing_pct = (missing_counts / len(df)) * 100
    
    missing_summary = pd.DataFrame({
        'Missing_Count': missing_counts,
        'Missing_Percentage': missing_pct,
        'Data_Type': df.dtypes
    }).sort_values('Missing_Count', ascending=False)
    
    # Show only columns with missing values
    has_missing = missing_summary[missing_summary['Missing_Count'] > 0]
    
    if len(has_missing) > 0:
        print(f"Columns with missing values: {len(has_missing)}")
        print(has_missing)
        
        # Categorize missing patterns
        high_missing = has_missing[has_missing['Missing_Percentage'] > 50]
        medium_missing = has_missing[(has_missing['Missing_Percentage'] > 5) & (has_missing['Missing_Percentage'] <= 50)]
        low_missing = has_missing[has_missing['Missing_Percentage'] <= 5]
        
        print(f"\nMISSING VALUE CATEGORIES:")
        print(f"├── High missing (>50%): {len(high_missing)} columns")
        print(f"├── Medium missing (5-50%): {len(medium_missing)} columns")
        print(f"└── Low missing (≤5%): {len(low_missing)} columns")
        
    else:
        print("✓ No missing values found!")
    
    return missing_summary

# Analyze missing values in master beneficiary dataset
missing_beneficiary = analyze_missing_values(df_beneficiary_master, "MASTER BENEFICIARY")

# Analyze missing values in inpatient claims
missing_inpatient = analyze_missing_values(df_inpatient, "INPATIENT CLAIMS")


MISSING VALUES ANALYSIS - MASTER BENEFICIARY
Columns with missing values: 1
               Missing_Count  Missing_Percentage Data_Type
BENE_DEATH_DT         338183           98.410855   float64

MISSING VALUE CATEGORIES:
├── High missing (>50%): 1 columns
├── Medium missing (5-50%): 0 columns
└── Low missing (≤5%): 0 columns

MISSING VALUES ANALYSIS - INPATIENT CLAIMS
Columns with missing values: 70
                        Missing_Count  Missing_Percentage Data_Type
HCPCS_CD_29                     66773          100.000000   float64
HCPCS_CD_30                     66773          100.000000   float64
HCPCS_CD_31                     66773          100.000000   float64
HCPCS_CD_32                     66773          100.000000   float64
HCPCS_CD_2                      66773          100.000000   float64
HCPCS_CD_1                      66773          100.000000   float64
HCPCS_CD_5                      66773          100.000000   float64
HCPCS_CD_6                      66773          100.0

### Block 6: Clean Missing Values in Beneficiary Data

In [6]:
def clean_beneficiary_missing_values(df):
    """
    Clean missing values in beneficiary data with business logic
    """
    print("CLEANING BENEFICIARY MISSING VALUES")
    print("="*40)
    
    df_clean = df.copy()
    original_rows = len(df_clean)
    
    # 1. Handle BENE_DEATH_DT (expected to be mostly missing for living patients)
    death_missing_before = df_clean['BENE_DEATH_DT'].isnull().sum()
    print(f"Death date missing: {death_missing_before:,} records (expected for living patients)")
    
    # 2. Check for any unexpected missing values in key demographic fields
    key_demographics = ['DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_SEX_IDENT_CD', 'BENE_RACE_CD']
    
    for col in key_demographics:
        if col in df_clean.columns:
            missing_count = df_clean[col].isnull().sum()
            if missing_count > 0:
                print(f"WARNING: {col} has {missing_count} missing values")
            else:
                print(f"✓ {col}: No missing values")
    
    # 3. Handle missing values in financial fields (fill with 0)
    financial_cols = [col for col in df_clean.columns if any(x in col for x in ['MEDREIMB', 'BENRES', 'PPPYMT'])]
    
    if financial_cols:
        print(f"\nCleaning {len(financial_cols)} financial columns...")
        for col in financial_cols:
            missing_before = df_clean[col].isnull().sum()
            if missing_before > 0:
                df_clean[col].fillna(0, inplace=True)
                print(f"  {col}: Filled {missing_before} missing values with 0")
    
    # 4. Handle missing values in coverage months (fill with 0)
    coverage_cols = [col for col in df_clean.columns if 'CVRAGE' in col or 'CVRG' in col]
    
    if coverage_cols:
        print(f"\nCleaning {len(coverage_cols)} coverage columns...")
        for col in coverage_cols:
            missing_before = df_clean[col].isnull().sum()
            if missing_before > 0:
                df_clean[col].fillna(0, inplace=True)
                print(f"  {col}: Filled {missing_before} missing values with 0")
    
    print(f"\n✓ Beneficiary data cleaning complete")
    print(f"  Rows before: {original_rows:,}")
    print(f"  Rows after: {len(df_clean):,}")
    
    return df_clean

# Clean beneficiary data
df_beneficiary_clean = clean_beneficiary_missing_values(df_beneficiary_master)

CLEANING BENEFICIARY MISSING VALUES
Death date missing: 338,183 records (expected for living patients)
✓ DESYNPUF_ID: No missing values
✓ BENE_BIRTH_DT: No missing values
✓ BENE_SEX_IDENT_CD: No missing values
✓ BENE_RACE_CD: No missing values

Cleaning 9 financial columns...

Cleaning 4 coverage columns...

✓ Beneficiary data cleaning complete
  Rows before: 343,644
  Rows after: 343,644


### Block 7: Clean Missing Values in Inpatient Data

In [7]:
def clean_inpatient_missing_values(df):
    """
    Clean missing values in inpatient claims with healthcare logic
    """
    print("CLEANING INPATIENT MISSING VALUES")
    print("="*40)
    
    df_clean = df.copy()
    original_rows = len(df_clean)
    
    # 1. Handle missing admission/discharge dates (critical for readmission analysis)
    print("Handling critical date fields...")
    
    date_fields = ['CLM_FROM_DT', 'CLM_THRU_DT', 'CLM_ADMSN_DT', 'NCH_BENE_DSCHRG_DT']
    for field in date_fields:
        if field in df_clean.columns:
            missing_count = df_clean[field].isnull().sum()
            if missing_count > 0:
                print(f"  WARNING: {field} has {missing_count} missing values")
                
                # For critical analysis, we may need to drop records with missing key dates
                if field in ['CLM_ADMSN_DT', 'NCH_BENE_DSCHRG_DT']:
                    print(f"    These records cannot be used for readmission analysis")
    
    # 2. Handle missing length of stay (derive from dates where possible)
    if 'CLM_UTLZTN_DAY_CNT' in df_clean.columns:
        los_missing = df_clean['CLM_UTLZTN_DAY_CNT'].isnull().sum()
        print(f"\nLength of stay missing: {los_missing} records")
        
        if los_missing > 0 and 'CLM_FROM_DT' in df_clean.columns and 'CLM_THRU_DT' in df_clean.columns:
            # Try to calculate LOS from claim dates where both are available
            mask = (df_clean['CLM_UTLZTN_DAY_CNT'].isnull() & 
                   df_clean['CLM_FROM_DT'].notna() & 
                   df_clean['CLM_THRU_DT'].notna())
            
            if mask.sum() > 0:
                # Calculate days between dates (YYYYMMDD format)
                from_dates = pd.to_datetime(df_clean.loc[mask, 'CLM_FROM_DT'], format='%Y%m%d', errors='coerce')
                thru_dates = pd.to_datetime(df_clean.loc[mask, 'CLM_THRU_DT'], format='%Y%m%d', errors='coerce')
                calculated_los = (thru_dates - from_dates).dt.days + 1  # +1 for same-day stays
                
                df_clean.loc[mask, 'CLM_UTLZTN_DAY_CNT'] = calculated_los
                print(f"  Calculated LOS for {calculated_los.notna().sum()} records from claim dates")
    
    # 3. Handle missing diagnosis codes (keep as missing - clinically meaningful)
    diagnosis_cols = [col for col in df_clean.columns if col.startswith('ICD9_DGNS_CD_')]
    if diagnosis_cols:
        print(f"\nDiagnosis codes: {len(diagnosis_cols)} columns")
        primary_dx_missing = df_clean['ICD9_DGNS_CD_1'].isnull().sum() if 'ICD9_DGNS_CD_1' in df_clean.columns else 0
        print(f"  Primary diagnosis missing: {primary_dx_missing} records")
        if primary_dx_missing > 0:
            print(f"    These records may need special handling in analysis")
    
    # 4. Handle missing financial amounts (fill with 0)
    financial_cols = [col for col in df_clean.columns if any(x in col for x in ['CLM_PMT_AMT', 'DDCTBL', 'COINSRNC'])]
    if financial_cols:
        print(f"\nCleaning {len(financial_cols)} financial columns...")
        for col in financial_cols:
            missing_before = df_clean[col].isnull().sum()
            if missing_before > 0:
                df_clean[col].fillna(0, inplace=True)
                print(f"  {col}: Filled {missing_before} missing values with 0")
    
    # 5. Summary of critical missing data for readmission analysis
    print(f"\nCRITICAL FIELDS FOR READMISSION ANALYSIS:")
    critical_fields = ['CLM_ADMSN_DT', 'NCH_BENE_DSCHRG_DT', 'DESYNPUF_ID']
    complete_records = len(df_clean)
    
    for field in critical_fields:
        if field in df_clean.columns:
            missing = df_clean[field].isnull().sum()
            complete_records = min(complete_records, len(df_clean) - missing)
            print(f"  {field}: {len(df_clean) - missing:,} complete records")
    
    print(f"\n✓ Records usable for readmission analysis: {complete_records:,}")
    print(f"✓ Inpatient data cleaning complete")
    
    return df_clean

# Clean inpatient data
df_inpatient_clean = clean_inpatient_missing_values(df_inpatient)

CLEANING INPATIENT MISSING VALUES
Handling critical date fields...

Length of stay missing: 68 records

Diagnosis codes: 10 columns
  Primary diagnosis missing: 95 records
    These records may need special handling in analysis

Cleaning 4 financial columns...
  NCH_BENE_IP_DDCTBL_AMT: Filled 2178 missing values with 0

CRITICAL FIELDS FOR READMISSION ANALYSIS:
  CLM_ADMSN_DT: 66,773 complete records
  NCH_BENE_DSCHRG_DT: 66,773 complete records
  DESYNPUF_ID: 66,773 complete records

✓ Records usable for readmission analysis: 66,773
✓ Inpatient data cleaning complete


### Block 8: Data Type Validation and Consistency

In [8]:
def validate_data_consistency():
    """
    Validate data consistency across combined datasets
    """
    print("DATA CONSISTENCY VALIDATION")
    print("="*40)
    
    # 1. Check beneficiary ID consistency
    beneficiary_ids = set(df_beneficiary_clean['DESYNPUF_ID'])
    inpatient_ids = set(df_inpatient_clean['DESYNPUF_ID'])
    
    print("BENEFICIARY ID CONSISTENCY:")
    print(f"├── Unique beneficiaries in master file: {len(beneficiary_ids):,}")
    print(f"├── Unique beneficiaries in inpatient: {len(inpatient_ids):,}")
    print(f"└── Inpatient IDs found in beneficiary: {len(inpatient_ids & beneficiary_ids):,}")
    
    orphaned_inpatient = inpatient_ids - beneficiary_ids
    if orphaned_inpatient:
        print(f"⚠️  WARNING: {len(orphaned_inpatient)} inpatient beneficiaries not in master file")
    else:
        print("✓ All inpatient beneficiaries found in master file")
    
    # 2. Validate date formats and ranges
    print(f"\nDATE VALIDATION:")
    
    # Check admission/discharge date consistency in inpatient data
    if 'CLM_ADMSN_DT' in df_inpatient_clean.columns and 'NCH_BENE_DSCHRG_DT' in df_inpatient_clean.columns:
        # Find records where discharge is before admission
        invalid_dates = df_inpatient_clean[
            (df_inpatient_clean['CLM_ADMSN_DT'].notna()) & 
            (df_inpatient_clean['NCH_BENE_DSCHRG_DT'].notna()) &
            (df_inpatient_clean['NCH_BENE_DSCHRG_DT'] < df_inpatient_clean['CLM_ADMSN_DT'])
        ]
        
        if len(invalid_dates) > 0:
            print(f"⚠️  WARNING: {len(invalid_dates)} records with discharge before admission")
        else:
            print("✓ All discharge dates are after admission dates")
    
    # 3. Check for reasonable date ranges (2008-2010)
    if 'CLM_ADMSN_DT' in df_inpatient_clean.columns:
        min_date = df_inpatient_clean['CLM_ADMSN_DT'].min()
        max_date = df_inpatient_clean['CLM_ADMSN_DT'].max()
        print(f"Admission date range: {min_date} to {max_date}")
        
        # Check for dates outside expected range
        outside_range = df_inpatient_clean[
            (df_inpatient_clean['CLM_ADMSN_DT'] < 20080101) | 
            (df_inpatient_clean['CLM_ADMSN_DT'] > 20101231)
        ]
        
        if len(outside_range) > 0:
            print(f"⚠️  WARNING: {len(outside_range)} records outside 2008-2010 range")
        else:
            print("✓ All dates within expected range (2008-2010)")
    
    # 4. Validate chronic condition consistency across years
    print(f"\nCHRONIC CONDITION CONSISTENCY:")
    chronic_conditions = [col for col in df_beneficiary_clean.columns if col.startswith('SP_') and col != 'SP_STATE_CODE']
    
    if chronic_conditions:
        print(f"Analyzing {len(chronic_conditions)} chronic conditions...")
        
        # Check for beneficiaries where chronic conditions "disappear" (should be rare)
        condition_changes = {}
        
        for condition in chronic_conditions[:3]:  # Check first 3 as example
            # Group by beneficiary and check if condition ever goes from True to False
            ben_condition = df_beneficiary_clean.groupby('DESYNPUF_ID')[condition].apply(list)
            
            reversals = 0
            for ben_id, values in ben_condition.items():
                if len(values) > 1:
                    # Check if any True is followed by False
                    for i in range(len(values)-1):
                        if values[i] == True and values[i+1] == False:
                            reversals += 1
                            break
            
            condition_changes[condition] = reversals
            print(f"  {condition}: {reversals} beneficiaries with condition reversals")

validate_data_consistency()

DATA CONSISTENCY VALIDATION
BENEFICIARY ID CONSISTENCY:
├── Unique beneficiaries in master file: 116,352
├── Unique beneficiaries in inpatient: 37,780
└── Inpatient IDs found in beneficiary: 37,780
✓ All inpatient beneficiaries found in master file

DATE VALIDATION:
✓ All discharge dates are after admission dates
Admission date range: 20071127 to 20101230
⚠️  WARNING: 226 records outside 2008-2010 range

CHRONIC CONDITION CONSISTENCY:
Analyzing 11 chronic conditions...
  SP_ALZHDMTA: 29542 beneficiaries with condition reversals
  SP_CHF: 33229 beneficiaries with condition reversals
  SP_CHRNKIDN: 23227 beneficiaries with condition reversals


### Block 9: Create Data Quality Report

In [9]:
def create_data_quality_report():
    """
    Generate comprehensive data quality report
    """
    print("DATA QUALITY REPORT")
    print("="*50)
    
    # Master dataset statistics
    print("MASTER BENEFICIARY DATASET:")
    print(f"├── Total records: {len(df_beneficiary_clean):,}")
    print(f"├── Unique beneficiaries: {df_beneficiary_clean['DESYNPUF_ID'].nunique():,}")
    print(f"├── Years covered: {sorted(df_beneficiary_clean['BENE_YEAR'].unique())}")
    print(f"└── Memory usage: {df_beneficiary_clean.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Inpatient dataset statistics  
    print(f"\nINPATIENT CLAIMS DATASET:")
    print(f"├── Total claims: {len(df_inpatient_clean):,}")
    print(f"├── Unique beneficiaries: {df_inpatient_clean['DESYNPUF_ID'].nunique():,}")
    print(f"├── Date range: {df_inpatient_clean['CLM_ADMSN_DT'].min()} to {df_inpatient_clean['NCH_BENE_DSCHRG_DT'].max()}")
    print(f"└── Memory usage: {df_inpatient_clean.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Data completeness for key fields
    print(f"\nDATA COMPLETENESS FOR KEY FIELDS:")
    
    # Beneficiary key fields
    ben_key_fields = ['DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_SEX_IDENT_CD', 'BENE_RACE_CD']
    for field in ben_key_fields:
        if field in df_beneficiary_clean.columns:
            completeness = (1 - df_beneficiary_clean[field].isnull().mean()) * 100
            print(f"├── Beneficiary {field}: {completeness:.1f}% complete")
    
    # Inpatient key fields
    inp_key_fields = ['DESYNPUF_ID', 'CLM_ADMSN_DT', 'NCH_BENE_DSCHRG_DT', 'ICD9_DGNS_CD_1']
    for field in inp_key_fields:
        if field in df_inpatient_clean.columns:
            completeness = (1 - df_inpatient_clean[field].isnull().mean()) * 100
            print(f"├── Inpatient {field}: {completeness:.1f}% complete")
    
    # Records ready for analysis
    print(f"\nREADINESS FOR READMISSION ANALYSIS:")
    
    # Count records with all required fields for readmission analysis
    required_fields = ['DESYNPUF_ID', 'CLM_ADMSN_DT', 'NCH_BENE_DSCHRG_DT']
    complete_mask = df_inpatient_clean[required_fields].notna().all(axis=1)
    complete_records = complete_mask.sum()
    
    print(f"├── Complete inpatient records: {complete_records:,} ({complete_records/len(df_inpatient_clean)*100:.1f}%)")
    print(f"├── Beneficiaries with complete data: {df_inpatient_clean[complete_mask]['DESYNPUF_ID'].nunique():,}")
    print(f"└── Ready for target variable creation: ✓")
    
    return complete_records

complete_records = create_data_quality_report()

DATA QUALITY REPORT
MASTER BENEFICIARY DATASET:
├── Total records: 343,644
├── Unique beneficiaries: 116,352
├── Years covered: [np.int64(2008), np.int64(2009), np.int64(2010)]
└── Memory usage: 80.3 MB

INPATIENT CLAIMS DATASET:
├── Total claims: 66,773
├── Unique beneficiaries: 37,780
├── Date range: 20071127 to 20101231
└── Memory usage: 44.0 MB

DATA COMPLETENESS FOR KEY FIELDS:
├── Beneficiary DESYNPUF_ID: 100.0% complete
├── Beneficiary BENE_BIRTH_DT: 100.0% complete
├── Beneficiary BENE_SEX_IDENT_CD: 100.0% complete
├── Beneficiary BENE_RACE_CD: 100.0% complete
├── Inpatient DESYNPUF_ID: 100.0% complete
├── Inpatient CLM_ADMSN_DT: 100.0% complete
├── Inpatient NCH_BENE_DSCHRG_DT: 100.0% complete
├── Inpatient ICD9_DGNS_CD_1: 99.9% complete

READINESS FOR READMISSION ANALYSIS:
├── Complete inpatient records: 66,773 (100.0%)
├── Beneficiaries with complete data: 37,780
└── Ready for target variable creation: ✓


### Block 10: Save Master Datasets

In [10]:
def save_master_datasets():
    """
    Save cleaned and combined master datasets
    """
    print("SAVING MASTER DATASETS")
    print("="*30)
    
    # Create directory if needed
    os.makedirs(processed_path, exist_ok=True)
    
    # Save master beneficiary dataset
    ben_file_csv = f"{processed_path}beneficiary_master_clean.csv"
    ben_file_parquet = f"{processed_path}beneficiary_master_clean.parquet"
    
    df_beneficiary_clean.to_csv(ben_file_csv, index=False)
    df_beneficiary_clean.to_parquet(ben_file_parquet)
    
    print(f"✓ Beneficiary master dataset saved:")
    print(f"  ├── CSV: {ben_file_csv}")
    print(f"  └── Parquet: {ben_file_parquet}")
    
    # Save clean inpatient dataset
    inp_file_csv = f"{processed_path}inpatient_master_clean.csv"
    inp_file_parquet = f"{processed_path}inpatient_master_clean.parquet"
    
    df_inpatient_clean.to_csv(inp_file_csv, index=False)
    df_inpatient_clean.to_parquet(inp_file_parquet)
    
    print(f"✓ Inpatient master dataset saved:")
    print(f"  ├── CSV: {inp_file_csv}")
    print(f"  └── Parquet: {inp_file_parquet}")
    
    # Save data summary
    summary_file = f"{processed_path}data_combination_summary.txt"
    with open(summary_file, 'w') as f:
        f.write("DATA COMBINATION & PREPROCESSING SUMMARY\n")
        f.write("="*50 + "\n\n")
        f.write(f"Master Beneficiary Dataset:\n")
        f.write(f"- Total records: {len(df_beneficiary_clean):,}\n")
        f.write(f"- Unique beneficiaries: {df_beneficiary_clean['DESYNPUF_ID'].nunique():,}\n")
        f.write(f"- Years: {sorted(df_beneficiary_clean['BENE_YEAR'].unique())}\n\n")
        f.write(f"Inpatient Claims Dataset:\n")
        f.write(f"- Total claims: {len(df_inpatient_clean):,}\n")
        f.write(f"- Unique beneficiaries: {df_inpatient_clean['DESYNPUF_ID'].nunique():,}\n")
        f.write(f"- Complete records: {complete_records:,}\n\n")
        f.write(f"Ready for Phase C: Target Variable Creation\n")
    
    print(f"✓ Summary saved: {summary_file}")

save_master_datasets()

SAVING MASTER DATASETS
✓ Beneficiary master dataset saved:
  ├── CSV: data/processed/beneficiary_master_clean.csv
  └── Parquet: data/processed/beneficiary_master_clean.parquet
✓ Inpatient master dataset saved:
  ├── CSV: data/processed/inpatient_master_clean.csv
  └── Parquet: data/processed/inpatient_master_clean.parquet
✓ Summary saved: data/processed/data_combination_summary.txt


### Block 11: Phase B Summary

In [11]:
print("\n" + "="*60)
print("PHASE B COMPLETE: DATA COMBINATION & PREPROCESSING")
print("="*60)

print("✅ ACCOMPLISHMENTS:")
print("├── Combined 3 beneficiary files into master dataset")
print("├── Cleaned missing values with healthcare business logic")
print("├── Validated data consistency across files")
print("├── Optimized data types and memory usage")
print("└── Created analysis-ready datasets")

print(f"\n📊 FINAL DATASET STATISTICS:")
print(f"├── Master Beneficiaries: {len(df_beneficiary_clean):,} records")
print(f"├── Unique Beneficiaries: {df_beneficiary_clean['DESYNPUF_ID'].nunique():,}")
print(f"├── Inpatient Claims: {len(df_inpatient_clean):,} claims")
print(f"├── Claims with Complete Data: {complete_records:,}")
print(f"└── Beneficiaries with Claims: {df_inpatient_clean['DESYNPUF_ID'].nunique():,}")

print(f"\n🚀 NEXT STEPS:")
print("├── Ready for Notebook 3: Target Variable Creation")
print("├── Define 30-day readmission events")
print("├── Create readmission target variable")  
print("├── Calculate baseline readmission rates")
print("└── Validate business logic")

print(f"\n✓ Phase B Complete - Ready for Target Variable Creation!")


PHASE B COMPLETE: DATA COMBINATION & PREPROCESSING
✅ ACCOMPLISHMENTS:
├── Combined 3 beneficiary files into master dataset
├── Cleaned missing values with healthcare business logic
├── Validated data consistency across files
├── Optimized data types and memory usage
└── Created analysis-ready datasets

📊 FINAL DATASET STATISTICS:
├── Master Beneficiaries: 343,644 records
├── Unique Beneficiaries: 116,352
├── Inpatient Claims: 66,773 claims
├── Claims with Complete Data: 66,773
└── Beneficiaries with Claims: 37,780

🚀 NEXT STEPS:
├── Ready for Notebook 3: Target Variable Creation
├── Define 30-day readmission events
├── Create readmission target variable
├── Calculate baseline readmission rates
└── Validate business logic

✓ Phase B Complete - Ready for Target Variable Creation!
